# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [1]:
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
os.getcwd()

'C:\\Users\\salman_pc\\Documents\\DATASCIENCE(SPRINGBOARD)\\GuidedCapStone\\1588204778_GuidedCapstoneStep4_and_step5HL.ipynb'

In [3]:
path='C:\\Users\\salman_pc\\Documents\\DATASCIENCE(SPRINGBOARD)\\GuidedCapStone'
os.chdir(path)

In [4]:
os.getcwd()

'C:\\Users\\salman_pc\\Documents\\DATASCIENCE(SPRINGBOARD)\\GuidedCapStone'

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [20]:
df = pd.read_csv('./data/step3_output.csv',index_col=0)
df.head()

,Name,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
2,Hilltop Ski Area,Alaska,2090,294,1796,0,0.0,0,0,0,...,30.0,30.0,150.000000,36.0,69.0,30.0,34.00000,152.0,30.0,0
4,Sunrise Park Resort,Arizona,11100,1800,9200,0,0.0,0,1,2,...,800.0,80.0,115.000000,49.0,250.0,74.0,78.00000,104.0,80.0,1
5,Yosemite Ski & Snowboard Area,California,7800,600,7200,0,0.0,0,0,0,...,88.0,0.0,110.000000,84.0,300.0,47.0,47.00000,107.0,0.0,1
8,Boreal Mountain Resort,California,7700,500,7200,0,0.0,0,1,1,...,380.0,200.0,150.000000,54.0,400.0,49.0,64.16681,150.0,200.0,1
9,Dodge Ridge,California,8200,1600,6600,0,0.0,0,0,1,...,862.0,0.0,115.103943,69.0,350.0,78.0,78.00000,140.0,0.0,1


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [21]:
dfo=df.select_dtypes(include=['object']) # select object type columns
#df = pd.concat([df.drop(dfo, axis=1), pd.get_dummies(dfo)], axis=1)
df.drop(dfo,axis=1,inplace=True)

## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [22]:
df.head()

,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
2,2090,294,1796,0,0.0,0,0,0,1,0,...,30.0,30.0,150.000000,36.0,69.0,30.0,34.00000,152.0,30.0,0
4,11100,1800,9200,0,0.0,0,1,2,3,1,...,800.0,80.0,115.000000,49.0,250.0,74.0,78.00000,104.0,80.0,1
5,7800,600,7200,0,0.0,0,0,0,1,3,...,88.0,0.0,110.000000,84.0,300.0,47.0,47.00000,107.0,0.0,1
8,7700,500,7200,0,0.0,0,1,1,3,1,...,380.0,200.0,150.000000,54.0,400.0,49.0,64.16681,150.0,200.0,1
9,8200,1600,6600,0,0.0,0,0,1,2,5,...,862.0,0.0,115.103943,69.0,350.0,78.0,78.00000,140.0,0.0,1


In [36]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df.drop(['AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df['AdultWeekend']

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [37]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train,X_test,y_train,y_test = train_test_split(X_scaled,y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [38]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [39]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [12]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model
explained_variance_score(y_test,y_pred)

0.5916620993196722

In [13]:
mean_absolute_error(y_test,y_pred)

7.051296509187421

**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [14]:
lm.intercept_

56.52144324206036

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [15]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])


,Coefficient
summit_elev,0.649325
vertical_drop,1.298404
base_elev,0.701358
trams,0.363931
fastEight,0.946999
...,...
state_Virginia,0.871356
state_Washington,0.355133
state_West Virginia,1.058878
state_Wisconsin,0.371071


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [27]:
explained_variance_score(y_test,y_pred)

0.4937747032772092

In [28]:
mean_absolute_error(y_test,y_pred)

7.617727165276725

In [29]:
lm.intercept_

56.52477980767324

In [30]:
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])

,Coefficient
summit_elev,9.189255e+00
vertical_drop,8.296787e-01
base_elev,1.119857e+01
trams,1.998401e-14
fastEight,2.486900e-14
fastSixes,1.221245e-14
fastQuads,2.091381e-01
quad,1.750399e+00
triple,8.704917e-02
double,5.288194e-01


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [31]:
df.head()

,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
2,2090,294,1796,0,0.0,0,0,0,1,0,...,30.0,30.0,150.000000,36.0,69.0,30.0,34.00000,152.0,30.0,0
4,11100,1800,9200,0,0.0,0,1,2,3,1,...,800.0,80.0,115.000000,49.0,250.0,74.0,78.00000,104.0,80.0,1
5,7800,600,7200,0,0.0,0,0,0,1,3,...,88.0,0.0,110.000000,84.0,300.0,47.0,47.00000,107.0,0.0,1
8,7700,500,7200,0,0.0,0,1,1,3,1,...,380.0,200.0,150.000000,54.0,400.0,49.0,64.16681,150.0,200.0,1
9,8200,1600,6600,0,0.0,0,0,1,2,5,...,862.0,0.0,115.103943,69.0,350.0,78.0,78.00000,140.0,0.0,1


In [33]:
df.drop(df[['summit_elev']],axis=1,inplace=True)

In [34]:
df.head()

,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,double,surface,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
2,294,1796,0,0.0,0,0,0,1,0,2,...,30.0,30.0,150.000000,36.0,69.0,30.0,34.00000,152.0,30.0,0
4,1800,9200,0,0.0,0,1,2,3,1,0,...,800.0,80.0,115.000000,49.0,250.0,74.0,78.00000,104.0,80.0,1
5,600,7200,0,0.0,0,0,0,1,3,1,...,88.0,0.0,110.000000,84.0,300.0,47.0,47.00000,107.0,0.0,1
8,500,7200,0,0.0,0,1,1,3,1,2,...,380.0,200.0,150.000000,54.0,400.0,49.0,64.16681,150.0,200.0,1
9,1600,6600,0,0.0,0,0,1,2,5,4,...,862.0,0.0,115.103943,69.0,350.0,78.0,78.00000,140.0,0.0,1


In [35]:
df.drop(df[['base_elev']],axis=1,inplace=True)
df.head()

,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,surface,total_chairs,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
2,294,0,0.0,0,0,0,1,0,2,3,...,30.0,30.0,150.000000,36.0,69.0,30.0,34.00000,152.0,30.0,0
4,1800,0,0.0,0,1,2,3,1,0,7,...,800.0,80.0,115.000000,49.0,250.0,74.0,78.00000,104.0,80.0,1
5,600,0,0.0,0,0,0,1,3,1,5,...,88.0,0.0,110.000000,84.0,300.0,47.0,47.00000,107.0,0.0,1
8,500,0,0.0,0,1,1,3,1,2,8,...,380.0,200.0,150.000000,54.0,400.0,49.0,64.16681,150.0,200.0,1
9,1600,0,0.0,0,0,1,2,5,4,12,...,862.0,0.0,115.103943,69.0,350.0,78.0,78.00000,140.0,0.0,1


In [40]:
explained_variance_score(y_test,y_pred)

0.5109179193482354

In [41]:
mean_absolute_error(y_test,y_pred)

7.562534216563228

In [42]:
lm.intercept_

56.3569117941398

In [43]:
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])

,Coefficient
vertical_drop,3.040650e+00
trams,2.220446e-15
fastEight,4.440892e-16
fastSixes,3.330669e-15
fastQuads,3.851294e-01
quad,1.818826e+00
triple,1.554961e-01
double,5.160494e-01
surface,6.964727e-01
total_chairs,3.942655e-01


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0. |  |-|
| Model 2. | 0.| |'state'|
| Model 3. | 0. | |'state','summit_elev','base_elev'|

Model Selection: